In [1]:
import pandas as pd
import numpy as np
import sweetviz
#from AutoClean import AutoClean
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
#from clusteval import clusteval


In [18]:
'''
Objective: Maximize the Sales 
Constraints: Minimize the Customer Retention
Success Criteria: 
Business Success Criteria: Increase the Sales by 10% to 12% by targeting cross-selling opportunities on current customers.
ML Success Criteria: Achieve a Silhouette coefficient of at least 0.6
Economic Success Criteria: The insurance company will see an increase in revenues by at least 8% 

'''
auto = pd.read_csv('AutoInsurance.csv')
auto

Customer       State  Customer Lifetime Value Response  Coverage  \
0     BU79786  Washington              2763.519279       No     Basic   
1     QZ44356     Arizona              6979.535903       No  Extended   
2     AI49188      Nevada             12887.431650       No   Premium   
3     WW63253  California              7645.861827       No     Basic   
4     HB64268  Washington              2813.692575       No     Basic   
...       ...         ...                      ...      ...       ...   
9129  LA72316  California             23405.987980       No     Basic   
9130  PK87824  California              3096.511217      Yes  Extended   
9131  TD14365  California              8163.890428       No  Extended   
9132  UP19263  California              7524.442436       No  Extended   
9133  Y167826  California              2611.836866       No  Extended   

     Education Effective To Date EmploymentStatus Gender  Income  ...  \
0     Bachelor           2/24/11         Employed      F   56274  ...   
1     Bachelor           1/31/11       Unemployed      F       0  ...   
2     Bachelor           2/19/11         Employed      F   48767  ...   
3     Bachelor           1/20/11       Unemployed      M       0  ...   
4     Bachelor          3/2/2011         Employed      M   43836  ...   
...        ...               ...              ...    ...     ...  ...   
9129  Bachelor         10/2/2011         Employed      M   71941  ...   
9130   College         12/2/2011         Employed      F   21604  ...   
9131  Bachelor          6/2/2011       Unemployed      M       0  ...   
9132   College          3/2/2011         Employed      M   21941  ...   
9133   College           2/14/11       Unemployed      M       0  ...   

     Months Since Policy Inception Number of Open Complaints  \
0                                5                         0   
1                               42                         0   
2                               38                         0   
3                               65                         0   
4                               44                         0   
...                            ...                       ...   
9129                            89                         0   
9130                            28                         0   
9131                            37                         3   
9132                             3                         0   
9133                            90                         0   

      Number of Policies     Policy Type        Policy  Renew Offer Type  \
0                      1  Corporate Auto  Corporate L3            Offer1   
1                      8   Personal Auto   Personal L3            Offer3   
2                      2   Personal Auto   Personal L3            Offer1   
3                      7  Corporate Auto  Corporate L2            Offer1   
4                      1   Personal Auto   Personal L1            Offer1   
...                  ...             ...           ...               ...   
9129                   2   Personal Auto   Personal L1            Offer2   
9130                   1  Corporate Auto  Corporate L3            Offer1   
9131                   2  Corporate Auto  Corporate L2            Offer1   
9132                   3   Personal Auto   Personal L2            Offer3   
9133                   1  Corporate Auto  Corporate L3            Offer4   

      Sales Channel Total Claim Amount  Vehicle Class Vehicle Size  
0             Agent         384.811147   Two-Door Car      Medsize  
1             Agent        1131.464935  Four-Door Car      Medsize  
2             Agent         566.472247   Two-Door Car      Medsize  
3       Call Center         529.881344            SUV      Medsize  
4             Agent         138.130879  Four-Door Car      Medsize  
...             ...                ...            ...          ...  
9129            Web         198.234764  Four-Door Car      Medsize  
9130         Bran

In [3]:
auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9134 entries, 0 to 9133
Data columns (total 24 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Customer                       9134 non-null   object 
 1   State                          9134 non-null   object 
 2   Customer Lifetime Value        9134 non-null   float64
 3   Response                       9134 non-null   object 
 4   Coverage                       9134 non-null   object 
 5   Education                      9134 non-null   object 
 6   Effective To Date              9134 non-null   object 
 7   EmploymentStatus               9134 non-null   object 
 8   Gender                         9134 non-null   object 
 9   Income                         9134 non-null   int64  
 10  Location Code                  9134 non-null   object 
 11  Marital Status                 9134 non-null   object 
 12  Monthly Premium Auto           9134 non-null   i

In [4]:
auto.isna().sum()

Customer                         0
State                            0
Customer Lifetime Value          0
Response                         0
Coverage                         0
Education                        0
Effective To Date                0
EmploymentStatus                 0
Gender                           0
Income                           0
Location Code                    0
Marital Status                   0
Monthly Premium Auto             0
Months Since Last Claim          0
Months Since Policy Inception    0
Number of Open Complaints        0
Number of Policies               0
Policy Type                      0
Policy                           0
Renew Offer Type                 0
Sales Channel                    0
Total Claim Amount               0
Vehicle Class                    0
Vehicle Size                     0
dtype: int64

In [5]:
x = auto.drop([
    'Customer Lifetime Value',
    'Effective To Date',
    'Number of Open Complaints',
    'Number of Policies',
    'Policy Type',
    'Policy',
    'Renew Offer Type',
    'Vehicle Class',
    'Vehicle Size','Customer'
], axis = 1)

In [6]:
numeric_features = x.select_dtypes(exclude = 'object').columns
categorical_features = x.select_dtypes(include ='object').columns

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
#imputation
num_pipeline = Pipeline(steps = [('impute', SimpleImputer(strategy = 'mean'))])
num_preprocessor = ColumnTransformer(transformers=[('num', num_pipeline, numeric_features)])
impute_data = num_preprocessor.fit(x[numeric_features])
impute_df = pd.DataFrame(impute_data.transform(x), columns = numeric_features)

In [8]:
from feature_engine.outliers import Winsorizer
winsor = Winsorizer(capping_method = 'iqr', tail = 'both', fold = 1.5, variables = list(impute_df.columns))
outlier_pipeline = Pipeline(steps = [('wins', winsor)])
outlier_data = outlier_pipeline.fit(impute_df)
outlier_df = pd.DataFrame(outlier_data.transform(impute_df), columns = numeric_features)

In [9]:
scale_pipeline = Pipeline(steps =[('scale', MinMaxScaler())])
scale_preprocessor = ColumnTransformer(transformers=[('scale', scale_pipeline, outlier_df.columns)])
scale_data = scale_preprocessor.fit(outlier_df)
scale_df = pd.DataFrame(outlier_data.transform(outlier_df), columns = outlier_df.columns)

In [10]:

from sklearn.preprocessing import OneHotEncoder
encode_pipeline = Pipeline(steps = [('encode', OneHotEncoder(drop = 'first'))])
encode_preprocessor = ColumnTransformer(transformers=[('encode', encode_pipeline, categorical_features)])
encode_data = encode_preprocessor.fit(x[categorical_features])
encode_df = pd.DataFrame((encode_data.transform(x[categorical_features])).toarray())
encode_df.columns = list(encode_data.get_feature_names_out())

In [11]:
cleandata = pd.concat([scale_df, encode_df], axis = 1)

In [12]:
# Applying AgglomerativeClustering and grouping data into 3 clusters 
hc1 = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'complete')

In [13]:
y_hc1 = hc1.fit_predict(cleandata)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_agglomerative.py:1005: FutureWarning: Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead
  warnings.warn(


In [14]:
y_hc1

array([2, 0, 2, ..., 0, 0, 0], dtype=int64)

In [15]:
hc1.labels_

array([2, 0, 2, ..., 0, 0, 0], dtype=int64)

In [16]:
#cluster evaluation
from sklearn.metrics import silhouette_score
silhouette_score(cleandata, hc1.labels_)

0.537237316015739

In [17]:
Q1. Why is it important to define objectives for any business problem?
Q1. Defining objectives for any business problem is crucial as it provides clarity on desired outcomes and guides decision-making processes effectively.
Q2. How can the quality of a Machine Learning model developed for a business problem be maintained?
Q2. To maintain the quality of a Machine Learning model for a business problem, continuous monitoring, retraining with updated data, and incorporating feedback loops are essential.
Q3. What is the first document created for any ML project?
Q3. The first document created for any ML project is typically the project proposal or a project plan outlining the problem statement, objectives, methodologies, and expected outcomes.
Q4. How can data with multiple sheets be loaded?
Q4. To load data with multiple sheets, one can use libraries like pandas in Python, specifying sheet names or indices while reading the Excel file.
Q5. What are Auto EDA techniques?
Q5. Auto EDA techniques include automated exploratory data analysis methods such as pandas_profiling, Sweetviz, and DataPrep.
Q6. What are the four business moments, and what insights can we draw from them?
Q6. The four business moments are acquisition, growth, retention, and monetization. From these, insights on customer behavior, market trends, and revenue generation can be drawn.
Q7. What techniques are involved in data Pre-Processing?
Q7. Techniques in data pre-processing include data cleaning, normalization, transformation, handling missing values, and feature scaling.
Q8. When should label encoding and one-hot encoding be used?
Q8. Label encoding is used when categorical variables are ordinal, while one-hot encoding is preferred for nominal categorical variables.
Q9. What technique is used to remove outliers?
Q9. The technique to remove outliers involves statistical methods like Z-score, IQR (Interquartile Range), or visual methods like box plots.
Q10. What are the techniques to check whether data is normally distributed or not?
Q10. Techniques to check data normality include Shapiro-Wilk test, Kolmogorov-Smirnov test, and visual inspection like histograms or Q-Q plots.
Q12. How can data be made scale-free?
Q12. Making data scale-free involves techniques like normalization or standardization to ensure that variables are on a similar scale.
Q13. What types of graphs are used to depict bivariate analysis?
Q13. Scatter plots, bubble charts, heatmaps, and pair plots are commonly used to depict bivariate analysis.
Q14. What is meant by bivariate frequency distribution?
Q14. Bivariate frequency distribution refers to the distribution of two variables simultaneously, showing the frequency of their joint occurrences.
Q15. Which libraries are used in Hierarchical clustering?
Q15. Libraries used in Hierarchical clustering include scipy and scikit-learn in Python.
Q16. What is the difference between Agglomerative clustering and Divisive Clustering?
Q16. Agglomerative clustering starts with individual data points as clusters and merges them, while divisive clustering starts with one cluster and splits it recursively.
Q17. Which metric is used to find distance/similarities between two data points and between a record and a cluster?
Q17. The metric used to find distances/similarities between data points and clusters is Euclidean distance, Manhattan distance, or cosine similarity.
Q18. What parameters need to be passed to plot a Dendrogram?
Q18. Parameters needed to plot a Dendrogram include linkage method and the distance metric.
Q19. How is cluster evaluation performed? Which techniques are used?
Q19. Cluster evaluation is performed using techniques like silhouette analysis, Davies-Bouldin Index, and Calinski-Harabasz Index.
Q20. What do the Silhouette coefficient, Calinski Harbaz, and Davies-Bouldin Index indicate in hierarchical clustering?
Q20. The Silhouette coefficient measures how similar an object is to its cluster compared to other clusters, while Calinski-Harabasz and Davies-Bouldin Index indicate cluster compactness and separation in hierarchical clustering.

SyntaxError: invalid syntax (2338317550.py, line 2)